In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['D1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]
    Src['Datetime']                     = Query['datetime']
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']   = (Src['Close'] - Src['Open'])
    Src['Hilo']     = (Src['High']  - Src['Low'])
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
BINS = [100,200,500,800,1000,1200,1500,1800,2000,2200,2500,2800,3000,3200,3500,3800,4000,4200,4500,4800,5000,5500,6000,6500,7000,7500,8000]

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()

    for b in BINS:  Calc[f'Chg Abs > {b}']      = (Calc[f'Chg Abs'] > b).astype(int)
    for b in BINS:  Calc[f'Hilo > {b}']         = (Calc[f'Hilo']    > b).astype(int)
    
    for b in BINS:  Calc[f'Chg Abs > {b} +{1}'] =  Calc[f'Chg Abs > {b}'].shift(-1)
    for b in BINS:  Calc[f'Chg Abs > {b} +{2}'] =  Calc[f'Chg Abs > {b}'].shift(-2)
    for b in BINS:  Calc[f'Chg Abs > {b} +{3}'] =  Calc[f'Chg Abs > {b}'].shift(-3)

    for b in BINS:  Calc[f'Hilo > {b} +{1}']    =  Calc[f'Hilo > {b}'].shift(-1)
    for b in BINS:  Calc[f'Hilo > {b} +{2}']    =  Calc[f'Hilo > {b}'].shift(-2)
    for b in BINS:  Calc[f'Hilo > {b} +{3}']    =  Calc[f'Hilo > {b}'].shift(-3)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [ ]:
def STATS(Calc, Col):
    pipe = []
    for b in BINS:
        Df = Calc[Calc[f'{Col} > {b}']==1] #.reset_index(drop=1)
        pipe.append({ 
            f'{Col} > D0':  b,
            f'{Col} > D+1': np.round(Df[f'{Col} > {b} +{1}'].mean() *100, 1), 
            f'{Col} > D+2': np.round(Df[f'{Col} > {b} +{2}'].mean() *100, 1), 
            f'{Col} > D+3': np.round(Df[f'{Col} > {b} +{3}'].mean() *100, 1), 
        })
    pass
    return pd.DataFrame(pipe)

In [ ]:
STATS(Calc, Col='Chg Abs')

In [ ]:
STATS(Calc, Col='Hilo')

# Snippets

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()

    for b in BINS:  Calc[f'Chg Abs > {b}']      = (Calc[f'Chg Abs'] > b).astype(int)
    for b in BINS:  Calc[f'Hilo > {b}']         = (Calc[f'Hilo']    > b).astype(int)
    
    for b in BINS:  Calc[f'Chg Abs > {b} +{1}'] =  Calc[f'Chg Abs > {b}'].shift(-1)
    for b in BINS:  Calc[f'Chg Abs > {b} +{2}'] =  Calc[f'Chg Abs > {b}'].shift(-2)
    for b in BINS:  Calc[f'Chg Abs > {b} +{3}'] =  Calc[f'Chg Abs > {b}'].shift(-3)

    for b in BINS:  Calc[f'Hilo > {b} +{1}']    =  Calc[f'Hilo > {b}'].shift(-1)
    for b in BINS:  Calc[f'Hilo > {b} +{2}']    =  Calc[f'Hilo > {b}'].shift(-2)
    for b in BINS:  Calc[f'Hilo > {b} +{3}']    =  Calc[f'Hilo > {b}'].shift(-3)
    return Calc


def STATS(Calc, Col):
    pipe = []
    for b in BINS:
        Df = Calc[Calc[f'{Col} > {b}']==1] 
        pipe.append({ 
            f'{Col} > D0':  b,
            f'{Col} > D+1': np.round(Df[f'{Col} > {b} +{1}'].mean() *100, 1), 
            f'{Col} > D+2': np.round(Df[f'{Col} > {b} +{2}'].mean() *100, 1), 
            f'{Col} > D+3': np.round(Df[f'{Col} > {b} +{3}'].mean() *100, 1), 
        })
    pass
    return pd.DataFrame(pipe)


STATS(Calc, Col='Hilo')
STATS(Calc, Col='Chg Abs')